# MT Mount Characterization

From Craig Lage's notebook collection, edited to look at the TMA offset tests on the night of 3/9/2023

In [6]:
import sys, time, os, asyncio, glob
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

import pickle as pkl
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.sitcom.vandv.efd import create_efd_client

ModuleNotFoundError: No module named 'lsst.sitcom'

In [2]:
client = create_efd_client()

NameError: name 'create_efd_client' is not defined

In [ ]:

async def az_el(start, end):
    ''' 
    Loads in data from mount encoders for a given time range
    Returns azimuth and elevation encoder positions, times when the mount is tracking the target, and 
    information about whether the azimuth and elevation is in position:
    
    returns:
    
    az: dataframe with columns for mount encoder azimuth position and timestamps
    el: dataframe with columns for mount encoder elevation position and timestamps
    
    az_track: dataframe with commanded azimuth and time
    el_track: dataframe with commanded elevation and time
    
    azPos: dataframe with information about when the TMA is in position in azimuth
    elPos: dataframe with information about when the TMA is in position in elevation

    slew_starts: dataframe with times and ra 
    '''
    az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'timestamp'],  start, end)
    el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition', 'timestamp'],  start, end)    

    az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
    el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)  
    
    azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
    azPos = azPos[azPos['inPosition']] # Select only the True values
    elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
    elPos = elPos[elPos['inPosition']] # Select only the True values
    
    slew_starts = await client.select_time_series('lsst.sal.MTPtg.command_raDecTarget', \
                                            ['ra', 'declination', 'private_sndStamp'],  start, end)
    
    
    return az, el, az_track, el_track, azPos, elPos, slew_starts


def slew_settle_verification(az, el, az_track, el_track, azPos, elPos, slew_starts, plot = True, plot_all = False):
    ''' checking slew and settle for the 3.5 arcsec shifts and also more generally for larger shifts 
    Takes as input all outputs of the function az_el
    
    
    '''
    
    # plot azimuth and elevation for the whole interval
    plt.figure()
    az['actualPosition'].plot()
    plt.ylabel('Azimuth (Degrees)')
    plt.show()
    plt.figure()
    el['actualPosition'].plot()
    plt.ylabel('Elevation (Degrees)')
    plt.show()
    
    # find out when the slew is occurring
    azs = az_track.values[:,0]
    els = el_track.values[:,0]
       
    times = az_track.values[:,1]
    start_slew_times_1 = []
    elevations_1 = []
    slew_dist_1 = []
    slew_dist_az_1 = []
    slew_dist_el_1 = []
    slew_times_1 = []
        
    for i in range(1,len(az_track)):
        az_shift = abs(azs[i] - azs[i-1])
        el_shift = abs(els[i] - els[i-1])
        if (az_shift > 0.1) or (el_shift > 0.1):
            start_slew_times_1.append(times[i])
            elevations_1.append(els[i])
            az_shift_mod = az_shift * np.cos(els[i]*np.pi/180.0)
            shift = np.sqrt(el_shift*el_shift + az_shift_mod*az_shift_mod)
            slew_dist_az_1.append(az_shift_mod)
            slew_dist_el_1.append(el_shift)
            slew_dist_1.append(shift)
    # print(len(start_slew_times_1))

    # Now in position timestamps

    inPos_1 = []
    azPosValues = azPos.values[:,1]
    elPosValues = elPos.values[:,1]

    for i in range(len(azPos)):
        if azPosValues[i] > elPosValues[i]:
            inPos_1.append(azPosValues[i])
        else:
            inPos_1.append(elPosValues[i])

    # Now pair up slew starts and in position timestamps

    pairMin = 1.0
    pairMax = 10.0

    start_slew_times = []
    slew_dist = []
    slew_dist_az = []
    slew_dist_el = []
    slew_times = []
    inPos = []
    elevations = []

    for i in range(len(start_slew_times_1)):
        for j in range(len(inPos_1)):
            deltaT = inPos_1[j] - start_slew_times_1[i] 
            if deltaT > pairMin and deltaT < pairMax:
                inPos.append(inPos_1[j])
                start_slew_times.append(start_slew_times_1[i])
                elevations.append(elevations_1[i])
                slew_times.append(deltaT)
                slew_dist_az.append(slew_dist_az_1[i])
                slew_dist_el.append(slew_dist_el_1[i])
                slew_dist.append(slew_dist_1[i])
        
    print(len(inPos), len(start_slew_times), len(slew_times), len(slew_dist))
    
    slew_dist = np.array(slew_dist)
    slew_dist_az = np.array(slew_dist_az)
    slew_dist_el = np.array(slew_dist_el)
    slew_times = np.array(slew_times)
    elevations = np.array(elevations)
    
    time_deltas = np.zeros(len(start_slew_times))
    
    # add in the time between when the command was sent and when the slew started so we have the total time
    # note: this allows us to keep the time deltas between command sent and slew start, but it's more efficient
    # to just use the slew_starts time stamp.
    
    for i, start_slew_time in enumerate(start_slew_times):
        k = np.argmin(np.abs((Time(start_slew_time, format='unix_tai', scale='utc') - \
                              Time(slew_starts.private_sndStamp, format='unix_tai', scale='utc')).to_value('s')))
        time_delta = (Time(start_slew_time, format='unix_tai', scale='utc') - \
                      Time(slew_starts.private_sndStamp[k], format='unix_tai', scale='utc'))
        time_deltas[i] = time_delta.to_value('s')
        
    
    slew_times = slew_times + time_deltas
    
    if plot:
        plot_slew_settle(slew_times, slew_dist, start_slew_times, elevations, slew_dist_az, slew_dist_el, time_deltas)
        
        delta1 = 0
        delta = 240
        azPos_values = azPos.values[:,1]
        plt.subplot(1,1,1)
        plt.title("Azimuth Slew and Tracking")
        ax1 = az['actualPosition'].plot(color='red')
        for start_slew_time in start_slew_times:
            ss_time = Time(start_slew_time, format='unix_tai', scale='utc').isot  
            ax1.axvline(ss_time, color="black", linestyle="--")
        for inP in inPos:
            ip_time = Time(inP, format='unix_tai', scale='utc').isot  
            ax1.axvline(ip_time, color="blue", linestyle="--")
        for i in range(len(MTPtg.index)):
            start_time = Time(MTPtg.index[i], scale='utc').isot  
            ax1.axvline(start_time, color="red", linestyle="--")
        ax1.set_xlim((start+TimeDelta(delta1, format='sec')).isot, (start+TimeDelta(delta, format='sec')).isot)
        ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
        ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
        # ax1.set_ylim(-95, -75)
        ax1.legend()
        plt.show()
    
    '''
    
    for i, slew_time in enumerate(slew_times):
        if elevations[i] < 60:
            
            delta1 = 0
            delta = 240
            plt.subplot(1,1,1)
            plt.title(r'Azimuth Slew and Tracking: Slew time is ' + str(round(slew_time, 2)) + ' s')
            ax1 = az['actualPosition'].plot(color='red')
            ss_time = Time(start_slew_times[i], format='unix_tai', scale='utc').isot  
            ax1.axvline(ss_time, color="black", linestyle="--")
            ip_time = Time(inPos[i], format='unix_tai', scale='utc').isot  
            ax1.axvline(ip_time, color="blue", linestyle="--")
            start_time = Time(MTPtg.index[i], scale='utc')  
            ax1.axvline(start_time.isot, color="red", linestyle="--")
            
            ax1.set_xlim((start_time).isot, (start_time+TimeDelta(5, format='sec')).isot)
            ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
            ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
            ax1.set_ylim(135, 145)
            ax1.legend()            
            plt.show()
            
            plt.subplot(1,1,1)
            plt.title(r'Elevation Slew and Tracking: Slew time is ' + str(round(slew_time ,2)) + ' s')
            ax1 = el['actualPosition'].plot(color='red')
            ss_time = Time(start_slew_times[i], format='unix_tai', scale='utc').isot  
            ax1.axvline(ss_time, color="black", linestyle="--")
            ip_time = Time(inPos[i], format='unix_tai', scale='utc').isot  
            ax1.axvline(ip_time, color="blue", linestyle="--")
            start_time = Time(MTPtg.index[i], scale='utc')
            ax1.axvline(start_time.isot, color="red", linestyle="--")
            
            ax1.set_xlim((start_time).isot, (start_time+TimeDelta(5, format='sec')).isot)
            ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
            ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
            ax1.set_ylim(30, 40)
            ax1.legend()            
            plt.show()
            '''
    
    return start_slew_times, slew_times, slew_dist, slew_dist_az, slew_dist_el, time_deltas, inPos, elevations

def plot_slew_settle(slew_times, slew_dist, start_slew_times, elevations, slew_dist_az, slew_dist_el, time_deltas):
    '''Plot slew and settle histograms'''
    plt.hist(time_deltas)
    plt.xlabel('Time between MTPtg command send and slew start (s)')
    plt.show()
    
    
    plt.subplots_adjust(wspace=0.5)
    plt.subplot(1,2,1)
    plt.hist(slew_times[(3 < slew_dist) & (slew_dist < 4)  & (elevations < 60)], alpha = 0.7, label = 'El < 60')
    plt.hist(slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], alpha = 0.7, label = 'El > 60')
    plt.xlabel("Slew and settle time (seconds)")
    plt.legend()
    # plt.xlim(0.0, 10.0)
    plt.subplot(1,2,2)
    plt.scatter(slew_dist[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
                slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)])
    plt.scatter(slew_dist[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
                slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)])
    plt.ylabel("Slew and settle time (sec)")
    plt.xlabel("Slew distance (degrees)")
    plt.show()
    
    plt.subplots_adjust(wspace=0.5)
    plt.subplot(1,2,1)
    plt.title('3.5 degree slews')
    plt.scatter(slew_dist_el[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
                slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)])
    plt.scatter(slew_dist_el[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
                slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)])
    plt.ylabel("Slew and settle time (sec)")
    plt.xlabel("Elevation slew distance (degrees)")
    plt.legend()
    # plt.xlim(0.0, 10.0)
    plt.subplot(1,2,2)
    plt.scatter(slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
                slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)])
    plt.scatter(slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
                slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)])
    plt.ylabel("Slew and settle time (sec)")
    plt.xlabel("Azimuth slew distance (degrees)")
    plt.show()
    
    plt.subplots_adjust(wspace=0.5)
    plt.subplot(1,2,1)
    plt.hist(slew_times[(elevations < 60)], alpha = 0.7, label = 'El < 60')
    plt.hist(slew_times[(elevations > 60)], alpha = 0.7, label = 'El > 60')
    plt.legend()
    plt.xlabel("Slew and settle time (seconds)")
    # plt.xlim(0.0, 10.0)
    plt.subplot(1,2,2)
    plt.scatter(slew_dist[(elevations < 60)], slew_times[(elevations < 60)])
    plt.scatter(slew_dist[(elevations > 60)], slew_times[(elevations > 60)])
    plt.ylabel("Slew and settle time(sec)")
    plt.xlabel("Slew distance (degrees)")
    plt.show()
    
def jitter_verification(az, el, start_slew_times, slew_times, inPos, delta = 32):
    ''' checking jitter for the images 
    delta is how many seconds over which to check the jitter
    '''
    
    
    azRmsVals = []
    elRmsVals = []
    imRmsVals = []
    
    for index in range(len(inPos) - 1):
        fig = plt.figure(figsize = (8,8))
        end_time = Time(inPos[index], format='unix_tai', scale='utc') + timedelta(seconds = delta)

        end_time = end_time.utc
        
        plotAz = az[(az['timestamp'] > inPos[index]) & (az['timestamp'] < end_time.to_value('unix_tai'))]
        plotEl = el[(el['timestamp'] > inPos[index]) & (el['timestamp'] < end_time.to_value('unix_tai'))]
        
        ss_time = end_time.isot
        ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot


        # Calculate the tracking errors
        az_vals = np.array(plotAz.values[:,0])
        el_vals = np.array(plotEl.values[:,0])
        times_az = plotAz.values[:,1]
        times_el = plotEl.values[:,1]
        # The fits are much better if the time variable
        # is centered in the interval
        time_delta_az = times_az[int(len(plotAz.values) / 2)]
        time_delta_el = times_el[int(len(plotEl.values) / 2)]
        fit_times_az = [(times_az[i]-time_delta_az) for i in range(len(times_az))]
        fit_times_el = [(times_el[i]-time_delta_el) for i in range(len(times_el))]

        # Fit with a polynomial
        az_fit = np.polyfit(fit_times_az, az_vals, 4)
        el_fit = np.polyfit(fit_times_el, el_vals, 4)
        az_model = np.polyval(az_fit, fit_times_az)
        el_model = np.polyval(el_fit, fit_times_el)

        # Errors in arcseconds
        az_error = (az_vals - az_model) * 3600
        el_error = (el_vals - el_model) * 3600
    
        # Drive velocities
        az_vel = (az_model[-1] - az_model[0]) / (fit_times_az[-1] - fit_times_az[0]) * 3600.0
        el_vel = (el_model[-1] - el_model[0]) / (fit_times_el[-1] - fit_times_el[0]) * 3600.0
    
        # Calculate RMS
        az_rms = np.sqrt(np.mean(az_error[np.abs(az_error) < 10 * np.std(az_error)] * az_error[np.abs(az_error) < 10 * np.std(az_error)]))
        el_rms = np.sqrt(np.mean(el_error[np.abs(el_error) < 10 * np.std(el_error)] * el_error[np.abs(el_error) < 10 * np.std(el_error)]))
        azRmsVals.append(az_rms)
        elRmsVals.append(el_rms)
        # Calculate Image impact RMS
        # We are less sensitive to Az errors near the zenith
        image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
        image_el_rms = el_rms
        imRmsVals.append(np.sqrt(image_az_rms*image_az_rms + image_el_rms*image_el_rms))
        
        plt.subplots_adjust(wspace=0.3, hspace=0.5)
        plt.suptitle(f"MT Mount Jitter - {ip_time}", fontsize = 18)
        plt.subplot(2,2,1)
        ax1 = plotAz['actualPosition'].plot(legend=True, color='red')
        ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
        ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
        ax1.set_title(f"Azimuth\nAve velocity={az_vel:.1f} arcsec/sec")
        ax1.set_ylabel("Degrees")
        ax1.legend()
        plt.subplot(2,2,2)
        ax3 = plotEl['actualPosition'].plot(legend=True, color='green')
        ax3.axvline(ss_time, color="black", linestyle="--", label="Start slew")
        ax3.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
        ax3.set_title(f"Elevation\nAve velocity={el_vel:.1f} arcsec/sec")
        ax3.set_ylabel("Degrees")
        ax3.legend()
        plt.subplot(2,2,3)
        plt.plot(fit_times_az, az_error, color='red')
        plt.title(f"Azimuth Residuals\n"
                  f"Azimuth RMS error = {az_rms:.3f} arcseconds\n"
                  f"  Image RMS error = {image_az_rms:.3f} arcseconds", fontsize=10)
        plt.ylim(-0.2,0.2)
        plt.xticks([])
        plt.ylabel("ArcSeconds")
        plt.subplot(2,2,4)
        plt.plot(fit_times_el, el_error, color='green')
        plt.title(f"Elevation residuals\n"
                  f"Elevation RMS error = {el_rms:.3f} arcseconds\n"
                  f"  Image RMS error = {image_el_rms:.3f} arcseconds", fontsize=10)
        plt.ylim(-0.2,0.2)
        plt.xticks([])
        plt.ylabel("ArcSeconds")
        timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
        plt.show()
            

    mount_data = {}
    mount_data['start_slew_times'] = start_slew_times
    mount_data['inPos'] = inPos
    mount_data['slew_times'] = slew_times
    # mount_data['slew_dist'] = slew_dist
    mount_data['azRmsVals'] = azRmsVals
    mount_data['elRmsVals'] = elRmsVals
    mount_data['imRmsVals'] = imRmsVals
    
    
    fig = plt.figure(figsize=(16,8))
    plt.suptitle("MT Mount RMS Jitter", fontsize = 18)
    azRmsVals = mount_data['azRmsVals']
    elRmsVals = mount_data['elRmsVals']
    imRmsVals = mount_data['imRmsVals']
    azMed = np.median(azRmsVals)
    elMed = np.median(elRmsVals)
    imMed = np.median(imRmsVals)
    plt.subplots_adjust(wspace=0.2)
    ax1 = plt.subplot(1,3,1)
    ax1.hist(azRmsVals)
    ax1.set_title(f"Azimuth RMS, N={len(azRmsVals)}")
    ax1.text(0.6,0.8, f"Median={azMed:.3f}", fontsize=12, transform=ax1.transAxes)
    # ax1.set_xlim(0,0.2)
    ax1.set_xlabel("RMS Jitter (arcseconds)")
    ax2 = plt.subplot(1,3,2)
    ax2.set_title(f"Elevation RMS, N={len(azRmsVals)}")
    ax2.hist(elRmsVals)
    ax2.text(0.5,0.8, f"Median={elMed:.3f}", fontsize=12, transform=ax2.transAxes)
    # ax2.set_xlim(0,0.2)
    ax2.set_xlabel("RMS Jitter (arcseconds)")
    ax3 = plt.subplot(1,3,3)
    ax3.set_title(f"Image Impact RMS, N={len(azRmsVals)}")
    ax3.hist(imRmsVals)
    ax3.text(0.6,0.8, f"Median={imMed:.3f}", fontsize=12, transform=ax3.transAxes)
    # ax3.set_xlim(0,0.2)
    ax3.set_xlabel("RMS Jitter (arcseconds)")
    plt.show()
    
    return azRmsVals, elRmsVals, imRmsVals    
    

In [ ]:
# This seems to work on the summit but not USDF
start = Time("2023-03-10T04:01:55", scale='utc')
end = Time("2023-03-10T04:09:58", scale='utc')
# first working 3.5 degree offset test
az, el, az_track, el_track, azPos, elPos, MTPtg = await az_el(start, end)
start_slew_times, slew_times, slew_dist, slew_dist_az, slew_dist_el, time_deltas, inPos, elevations = slew_settle_verification(az, el, az_track, el_track, azPos, elPos, MTPtg)
azRmsVals, elRmsVals, imRmsVals = jitter_verification(az, el, start_slew_times, slew_times, inPos, delta = 15)
df_0310_slew_1 = pd.DataFrame(data = {'Slew Time': slew_times,
                                 'Slew Dist': slew_dist, 
                                 'Slew Dist Az': slew_dist_az, 
                                 'Slew Dist El': slew_dist_el, 
                                 'Elevation': elevations})
df_0310_jitter_1 = pd.DataFrame(data = {'Az RMS': azRmsVals,
                                     'El RMS': elRmsVals,
                                     'Im RMS': imRmsVals})
df_0310_slew_1.to_csv('slew_settle_031023_1.csv')
df_0310_jitter_1.to_csv('jitter_031023_1.csv')

In [ ]:
# This seems to work on the summit but not USDF
start = Time("2023-03-10T04:13:02", scale='utc')
end = Time("2023-03-10T04:45:04", scale='utc')
# first working 3.5 degree offset test
az, el, az_track, el_track, azPos, elPos, MTPtg = await az_el(start, end)
start_slew_times, slew_times, slew_dist, slew_dist_az, slew_dist_el, inPos, elevations = slew_settle_verification(az, el, az_track, el_track, azPos, elPos, MTPtg)
azRmsVals, elRmsVals, imRmsVals = jitter_verification(az, el, start_slew_times, slew_times, inPos, delta = 15)
df_0310_slew_2 = pd.DataFrame(data = {'Slew Time': slew_times,
                                 'Slew Dist': slew_dist, 
                                 'Slew Dist Az': slew_dist_az, 
                                 'Slew Dist El': slew_dist_el, 
                                 'Elevation': elevations})
df_0310_jitter_2 = pd.DataFrame(data = {'Az RMS': azRmsVals,
                                     'El RMS': elRmsVals,
                                     'Im RMS': imRmsVals})
df_0310_slew_2.to_csv('slew_settle_031023_2.csv')
df_0310_jitter_2.to_csv('jitter_031023_2.csv')

In [ ]:
# random walk on 3/15
start = Time("2023-03-15T20:27:02", scale='utc')
end = Time("2023-03-15T21:10:04", scale='utc')
# first working 3.5 degree offset test
az, el, az_track, el_track, azPos, elPos, MTPtg = await az_el(start, end)
start_slew_times, slew_times, slew_dist, slew_dist_az, slew_dist_el, time_deltas, inPos, elevations = slew_settle_verification(az, el, az_track, el_track, azPos, elPos, MTPtg)
azRmsVals, elRmsVals, imRmsVals = jitter_verification(az, el, start_slew_times, slew_times, inPos, delta = 15)
df_0315_slew = pd.DataFrame(data = {'Slew Time': slew_times,
                                 'Slew Dist': slew_dist, 
                                 'Slew Dist Az': slew_dist_az, 
                                 'Slew Dist El': slew_dist_el, 
                                 'Elevation': elevations})
df_0315_jitter = pd.DataFrame(data = {'Az RMS': azRmsVals,
                                     'El RMS': elRmsVals,
                                     'Im RMS': imRmsVals})
df_0315_slew.to_csv('slew_settle_031523.csv')
df_0315_jitter.to_csv('jitter_031523.csv')

# Tests on 3/17

Relevant Rubin TV sequence numbers and times for this night:

1. 292 - 858, 07:37:22 - 09:49:22

In [ ]:
# offset grid on 3/17
start = Time("2023-03-18T07:37:00", scale='utc')
end = Time("2023-03-18T09:50:04", scale='utc')
# first working 3.5 degree offset test
az, el, az_track, el_track, azPos, elPos, MTPtg = await az_el(start, end)
start_slew_times, slew_times, slew_dist, slew_dist_az, slew_dist_el, time_deltas, inPos, elevations = slew_settle_verification(az, el, az_track, el_track, azPos, elPos, MTPtg)
azRmsVals, elRmsVals, imRmsVals = jitter_verification(az, el, start_slew_times, slew_times, inPos, delta = 15)
df_0317_slew = pd.DataFrame(data = {'Slew Time': slew_times,
                                 'Slew Dist': slew_dist, 
                                 'Slew Dist Az': slew_dist_az, 
                                 'Slew Dist El': slew_dist_el, 
                                 'Elevation': elevations})
df_0317_jitter = pd.DataFrame(data = {'Az RMS': azRmsVals,
                                     'El RMS': elRmsVals,
                                     'Im RMS': imRmsVals})
df_0317_slew.to_csv('slew_settle_031723.csv')
df_0317_jitter.to_csv('jitter_031723.csv')

# Tests on 3/24 - random walk

Relevant Rubin TV sequence numbers and times for this night:

Note: 81-82 is startracker

1. 83 - 171, 01:26:29 - 01:46:02
2. 172 - 268, 01:58:42 - 02:17:27
3. 269 - 795, 02:43:43 - 04:29:31
4. 796 - 1140, 04:33:48 - 05:54:59
5. 1142 - 1273, 06:03:54 - 06:32:05
6. 1274 - 1771, 06:43:26 - 08:23:51
7. 1772 - 1869, 08:28:08 - 08:50:25
8. 1871 - 2144, 09:03:46 - 09:57:17

In [ ]:

times = [["2023-03-25T01:26:29", "2023-03-25T01:46:02"],
         ["2023-03-25T01:58:42", "2023-03-25T02:17:27"],
         ["2023-03-25T02:43:43", "2023-03-25T04:29:31"],
         ["2023-03-25T04:33:48", "2023-03-25T05:54:59"],
         ["2023-03-25T06:03:54", "2023-03-25T06:32:05"],
         ["2023-03-25T06:43:26", "2023-03-25T08:23:51"],
         ["2023-03-25T08:28:08", "2023-03-25T08:50:25"],
         ["2023-03-25T09:03:46", "2023-03-25T09:57:17"]]

slew_times = []
slew_dist = []
slew_dist_az = []
slew_dist_el = []
azRmsVals = []
elRmsVals = []
imRmsVals = []
elevations = []

for i, time in enumerate(times):
    start = Time(time[0], scale='utc')
    end = Time(time[1], scale='utc')

    az, el, az_track, el_track, azPos, elPos, MTPtg = await az_el(start, end)
    start_slew_times, slew_times_temp, slew_dist_temp, slew_dist_az_temp, slew_dist_el_temp, time_deltas, inPos, elevations_temp = slew_settle_verification(az, el, az_track, el_track, azPos, elPos, MTPtg)
    azRmsVals_temp, elRmsVals_temp, imRmsVals_temp = jitter_verification(az, el, start_slew_times, slew_times_temp, inPos, delta = 15)
    
    if slew_times == []:
        slew_times = (slew_times_temp)
        slew_dist = (slew_dist_temp)
        slew_dist_az = (slew_dist_az_temp)
        slew_dist_el = (slew_dist_el_temp)
        azRmsVals = (azRmsVals_temp)
        elRmsVals = (elRmsVals_temp)
        imRmsVals = (imRmsVals_temp)
        elevations = (elevations_temp)
    else:
        slew_times = np.append(slew_times, slew_times_temp)
        slew_dist = np.append(slew_dist, slew_dist_temp)
        slew_dist_az = np.append(slew_dist_az, slew_dist_az_temp)
        slew_dist_el = np.append(slew_dist_el, slew_dist_el_temp)
        azRmsVals = np.append(azRmsVals, azRmsVals_temp)
        elRmsVals = np.append(elRmsVals, elRmsVals_temp)
        imRmsVals = np.append(imRmsVals, imRmsVals_temp)
        elevations = np.append(elevations, elevations_temp)
    
df_0324_slew = pd.DataFrame(data = {'Slew Time': slew_times,
                                 'Slew Dist': slew_dist, 
                                 'Slew Dist Az': slew_dist_az, 
                                 'Slew Dist El': slew_dist_el, 
                                 'Elevation': elevations})
df_0324_jitter = pd.DataFrame(data = {'Az RMS': azRmsVals,
                                     'El RMS': elRmsVals,
                                     'Im RMS': imRmsVals})
df_0324_slew.to_csv('slew_settle_032423.csv')
df_0324_jitter.to_csv('jitter_032423.csv')

# Combining datasets

In [ ]:
# could be combined for 3/24, here doing it for all datasets

jitter_031523 = pd.read_csv('jitter_031523.csv')
jitter_031723 = pd.read_csv('jitter_031723.csv')
jitter_032423 = pd.read_csv('jitter_032423.csv')

slew_settle_031523 = pd.read_csv('slew_settle_031523.csv')
slew_settle_031723 = pd.read_csv('slew_settle_031723.csv')
slew_settle_032423 = pd.read_csv('slew_settle_032423.csv')

slew_times = np.concatenate((slew_settle_031523['Slew Time'], slew_settle_031723['Slew Time'], slew_settle_032423['Slew Time']))
slew_dist = np.concatenate((slew_settle_031523['Slew Dist'], slew_settle_031723['Slew Dist'], slew_settle_032423['Slew Dist'])) 
slew_dist_az = np.concatenate((slew_settle_031523['Slew Dist Az'], slew_settle_031723['Slew Dist Az'], slew_settle_032423['Slew Dist Az'])) 
slew_dist_el = np.concatenate((slew_settle_031523['Slew Dist El'], slew_settle_031723['Slew Dist El'], slew_settle_032423['Slew Dist El']))
elevations = np.concatenate((slew_settle_031523['Elevation'], slew_settle_031723['Elevation'], slew_settle_032423['Elevation'])) 

azRmsVals = np.concatenate((jitter_031523['Az RMS'], jitter_031723['Az RMS'], jitter_032423['Az RMS'])) 
elRmsVals = np.concatenate((jitter_031523['El RMS'], jitter_031723['El RMS'], jitter_032423['El RMS'])) 
imRmsVals = np.concatenate((jitter_031523['Im RMS'], jitter_031723['Im RMS'], jitter_032423['Im RMS'])) 

plt.figure()
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.hist(slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], label = 'El < 60', alpha = 0.7)
plt.hist(slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], label = 'El > 60', alpha = 0.7)
plt.xlabel("Slew and settle time (seconds)")
plt.legend()
# plt.xlim(0.0, 10.0)
plt.subplot(1,2,2)
plt.scatter(slew_dist[(3 < slew_dist) & (slew_dist < 4)& (elevations < 60)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4)& (elevations < 60)])
plt.scatter(slew_dist[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], label = 'El > 60')
plt.ylabel("Slew and settle time (sec)")
plt.xlabel("Slew distance (degrees)")
plt.title('Slew and Settle')
plt.legend()
plt.show()
    
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.hist(slew_times[elevations < 60], label = 'El < 60', alpha = 0.7)
plt.hist(slew_times[elevations > 60], label = 'El > 60', alpha = 0.7)
plt.xlabel("Slew and settle time (seconds)")
plt.legend()
# plt.xlim(0.0, 10.0)
plt.subplot(1,2,2)
plt.scatter(slew_dist[ (elevations < 60)], slew_times[(elevations < 60)])
plt.scatter(slew_dist[elevations > 60], slew_times[elevations > 60], label = 'El > 60')
plt.legend()
plt.ylabel("Slew and settle time(sec)")
plt.xlabel("Slew distance (degrees)")
plt.title('Slew and Settle')
plt.show()

fig = plt.figure(figsize=(20,10))
plt.suptitle("MT Mount RMS Jitter", fontsize = 22)
azMed = np.median(azRmsVals)
elMed = np.median(elRmsVals)
imMed = np.median(imRmsVals)
plt.subplots_adjust(wspace=0.2)
ax1 = plt.subplot(1,3,1)
ax1.hist(np.asarray(azRmsVals))
ax1.set_title(f"Azimuth RMS, N={len(azRmsVals)}")
ax1.text(0.6,0.8, f"Median={azMed:.3f}", fontsize=18, transform=ax1.transAxes)
# ax1.set_xlim(0,0.2)
ax1.set_xlabel("RMS Jitter (arcseconds)")
ax2 = plt.subplot(1,3,2)
ax2.set_title(f"Elevation RMS, N={len(azRmsVals)}")
ax2.hist(np.asarray(elRmsVals[elRmsVals < 0.1]))
ax2.text(0.5,0.8, f"Median={elMed:.3f}", fontsize=18, transform=ax2.transAxes)
# ax2.set_xlim(0,0.2)
ax2.set_xlabel("RMS Jitter (arcseconds)")
ax3 = plt.subplot(1,3,3)
ax3.set_title(f"Image Impact RMS, N={len(azRmsVals)}")
ax3.hist(np.asarray(imRmsVals[imRmsVals < 0.1]))
ax3.text(0.6,0.8, f"Median={imMed:.3f}", fontsize=18, transform=ax3.transAxes)
ax3.ticklabel_format(axis = 'x', style = 'sci')
# ax3.set_xlim(0,1)
ax3.set_xlabel(r"RMS Jitter (arcseconds")
plt.show()

plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.title('3.5 degree slews')
plt.scatter(slew_dist_el[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], label = 'El < 60')
plt.scatter(slew_dist_el[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], label = 'El > 60')
plt.legend()
plt.ylabel("Slew and settle time (sec)")
plt.xlabel("Elevation slew distance (degrees)")
plt.legend()
# plt.xlim(0.0, 10.0)
plt.subplot(1,2,2)
plt.scatter(slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], label = 'El < 60')
plt.scatter(slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], label = 'El > 60')
plt.legend()
plt.ylabel("Slew and settle time (sec)")
plt.xlabel("Azimuth slew distance (degrees)")
plt.show()

plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.title('3.5 degree slews')
plt.scatter(slew_dist_el[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], label = 'El < 60')
# plt.scatter(slew_dist_el[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
#             slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], label = 'El > 60')
# plt.legend()
plt.ylabel("Slew and settle time (sec)")
plt.xlabel("Elevation slew distance (degrees)")
plt.legend()
# plt.xlim(0.0, 10.0)
plt.subplot(1,2,2)
plt.scatter(slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], label = 'El < 60')
# plt.scatter(slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
#             slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], label = 'El > 60')
# plt.legend()
plt.ylabel("Slew and settle time (sec)")
plt.xlabel("Azimuth slew distance (degrees)")
plt.show()

In [ ]:

plt.scatter(elevations[(3 < slew_dist) & (slew_dist < 4)], \
            slew_times[(3 < slew_dist) & (slew_dist < 4)])
# plt.scatter(slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
#             slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], label = 'El > 60')
# plt.legend()
plt.ylabel("Slew and settle time (sec)")
plt.xlabel("Elevation (degrees)")
plt.show()

In [ ]:
plt.figure(figsize = (8, 5))
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
cm = plt.cm.get_cmap('Purples')
plt.title('Elevation < 60')
ax1 = plt.scatter(slew_dist_el[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
            slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], \
            c = slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)], cmap = cm)
plt.colorbar(ax1)
plt.ylabel("Azimuth slew distance (sec)")
plt.xlabel("Elevation slew distance (degrees)")
# plt.xlim(0.0, 10.0)
plt.subplot(1,2,2)
plt.title('Elevation > 60')
ax2 = plt.scatter(slew_dist_el[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
            slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], \
            c = slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations > 60)], cmap = cm)
plt.colorbar(ax2)
plt.ylabel("Azimuth slew distance (sec)")
plt.xlabel("Elevation slew distance (degrees)")
plt.show()

In [ ]:
plt.figure()
cm = plt.cm.get_cmap('gray')
ax1 = plt.scatter(elevations[(3 < slew_dist) & (slew_dist < 4)], \
            slew_dist_az[(3 < slew_dist) & (slew_dist < 4)], \
            c = slew_times[(3 < slew_dist) & (slew_dist < 4)], cmap = cm)
cbar = plt.colorbar(ax1)
cbar.set_label('Slew time (s)', rotation=270)

plt.scatter(elevations[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60) & (slew_times > 4)], \
            slew_dist_az[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60) & (slew_times > 4)], \
            marker = 'x', color = 'r')

plt.ylabel("Azimuth slew distance (degrees)")
plt.xlabel("Elevation (degrees)")
plt.show()

In [ ]:
max(slew_times_temp[(3 < slew_dist_temp) & (slew_dist_temp < 4) & (elevations_temp < 60)])

In [ ]:
print(len(slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)]))
print(len(slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60) & (slew_times > 4)]))
print(17/409)

Find out what's going on with the really short (2.5 s) slews

In [ ]:
len(slew_times[(3 < slew_dist) & (slew_dist < 4) & (elevations < 60)])

In [ ]:
print(len(imRmsVals))
print(len(imRmsVals[imRmsVals > 0.01]))
print(205/639)

In [ ]:
plt.hist(imRmsVals[imRmsVals < 0.05])